In [1]:
import pandas as pd, numpy as np
from rdkit import Chem

In [2]:
from scipy import stats

In [3]:
import pandas as pd, numpy as np, re, os
from src.evaluation import Evaluator, compilerPatterns, ResultsFromMeanStdDF
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 12})

In [4]:
## This cell would need for testing phase - Ignore this
%load_ext autoreload
%autoreload 2

In [5]:
paths = [ os.path.join(dataset.name, experiment.name) 
         for dataset in os.scandir('experiments') if dataset.is_dir() and 'chembl' in dataset.name 
         for experiment in os.scandir(dataset.path)  ]

In [6]:
paths=[]
for dataset in os.scandir('experiments'):
    if not dataset.is_dir() or not 'chembl' in dataset.name:
        continue

    if 'chemblAug5' in dataset.name:
        for setup in os.scandir(dataset):
            if '2hl_512hu' in setup.name and '256bs' in setup.name:
                paths.append(setup.path)

    elif 'chembl' in dataset.name:
        for setup in os.scandir(dataset):
            if '2hl_512hu' in setup.name and '512bs' in setup.name:
                paths.append(setup.path)

In [7]:
paths

['experiments/chemblAug5_SELFIES-RNN/2hl_512hu_256bs_0.001lr',
 'experiments/chemblAug5_SMILES-RNN/2hl_512hu_256bs_0.001lr',
 'experiments/chemblAug5_fragSMILES-RNN/2hl_512hu_256bs_300es_0.001lr',
 'experiments/chembl_SELFIES-RNN/2hl_512hu_512bs_0.001lr',
 'experiments/chembl_SMILES-RNN/2hl_512hu_512bs_0.001lr',
 'experiments/chembl_fragSMILES-RNN/2hl_512hu_512bs_300es_0.001lr',
 'experiments/chemblAug5_t-SMILES-RNN/2hl_512hu_256bs_0.001lr',
 'experiments/chembl_t-SMILES-RNN/2hl_512hu_512bs_0.001lr']

In [8]:
evaluators = [Evaluator(path) for path in paths]

In [9]:
evaluators_table = pd.concat([evaluator.get_as_DFcell() for evaluator in evaluators ], ignore_index=True, axis=0)

In [10]:
evaluators_table = evaluators_table.drop(columns=['dataset','notation']).groupby(['aug','hl','hu','bs','lr']).agg(lambda x: x.dropna())

In [11]:
evaluators_table

,,,,,SELFIES,SMILES,fragSMILES,t-SMILES
aug,hl,hu,bs,lr,,,,
1,2,512,512,0.001,<src.evaluation.Evaluator object at 0x7fc84a76...,<src.evaluation.Evaluator object at 0x7fc84a76...,<src.evaluation.Evaluator object at 0x7fc84a74...,<src.evaluation.Evaluator object at 0x7fc84a5b...
5,2,512,256,0.001,<src.evaluation.Evaluator object at 0x7fc84a59...,<src.evaluation.Evaluator object at 0x7fc84a59...,<src.evaluation.Evaluator object at 0x7fc84a83...,<src.evaluation.Evaluator object at 0x7fc84a74...


In [12]:
evaluators_table.map(lambda x: x.load_novels())

,,,,,SELFIES,SMILES,fragSMILES,t-SMILES
aug,hl,hu,bs,lr,,,,
1,2,512,512,0.001,True,True,True,True
5,2,512,256,0.001,True,True,True,True


In [13]:
evaluators_table.map(lambda x: x.load_scaffolds())

,,,,,SELFIES,SMILES,fragSMILES,t-SMILES
aug,hl,hu,bs,lr,,,,
1,2,512,512,0.001,True,True,True,True
5,2,512,256,0.001,True,True,True,True


## scaffolds analysis

In [14]:
scaffDFs=pd.concat( [evaluator.getScaffoldResults() for evaluator in evaluators if not evaluator.novels is None], ignore_index=True)

[16:12:51] WARNING: not removing hydrogen atom without neighbors
[16:12:51] WARNING: not removing hydrogen atom without neighbors
[16:13:00] non-ring atom 1 marked aromatic


In [15]:
scaffDFs['unique%']=scaffDFs['unique']/scaffDFs['total']
scaffDFs['novel%']=scaffDFs['novel']/scaffDFs['unique']
scaffDFs['chiral%']=scaffDFs['chiral']/scaffDFs['novel']

In [16]:
scaffGrp=scaffDFs.drop(columns=['fold','epoch','T','amount','dataset']).groupby(['aug','notation','hl','hu','bs','lr'],as_index=True, dropna=False)

In [17]:
scaffStats=scaffGrp.describe().loc(axis=1)[ ( slice(None),['mean','std'] ) ]

In [18]:
scaffStats=ResultsFromMeanStdDF(scaffStats)

In [20]:
scaffStats

total         unique           novel  \
aug notation   hl hu  bs  lr                                                    
1   SELFIES    2  512 512 0.001   5964.0 ± 6.0  5570.0 ± 40.0   5160.0 ± 70.0   
    SMILES     2  512 512 0.001   5976.0 ± 8.0  5490.0 ± 80.0  4600.0 ± 100.0   
    fragSMILES 2  512 512 0.001   5985.0 ± 8.0  5500.0 ± 50.0   4440.0 ± 70.0   
    t-SMILES   2  512 512 0.001   5908.0 ± 9.0  5330.0 ± 50.0   4730.0 ± 40.0   
5   SELFIES    2  512 256 0.001   5978.0 ± 6.0  5620.0 ± 40.0   5240.0 ± 60.0   
    SMILES     2  512 256 0.001   5976.0 ± 5.0  5610.0 ± 70.0  5000.0 ± 100.0   
    fragSMILES 2  512 256 0.001   5978.0 ± 7.0  5510.0 ± 80.0  4400.0 ± 100.0   
    t-SMILES   2  512 256 0.001  5925.0 ± 10.0  5380.0 ± 40.0   4730.0 ± 40.0   

                                         chiral        unique%         novel%  \
aug notation   hl hu  bs  lr                                                    
1   SELFIES    2  512 512 0.001    760.0 ± 40.0  0.934 ± 0.007  0.925 ± 0.006   
    SMILES     2  512 512 0.001    820.0 ± 40.0    0.92 ± 0.01  0.838 ± 0.009   
    fragSMILES 2  512 512 0.001    750.0 ± 50.0  0.918 ± 0.007  0.809 ± 0.007   
    t-SMILES   2  512 512 0.001    760.0 ± 60.0  0.902 ± 0.007  0.888 ± 0.004   
5   SELFIES    2  512 256 0.001    870.0 ± 60.0  0.941 ± 0.006  0.931 ± 0.004   
    SMILES     2  512 256 0.001  1000.0 ± 100.0    0.94 ± 0.01     0.9 ± 0.01   
    fragSMILES 2  512 256 0.001    830.0 ± 80.0    0.92 ± 0.01    0.81 ± 0.01   
    t-SMILES   2  512 256 0.001    810.0 ± 50.0  0.908 ± 0.008  0.879 ± 0.005   

                                       chiral%  
aug notation   hl hu  bs  lr                    
1   SELFIES    2  512 512 0.001  0.148 ± 0.008  
    SMILES     2  512 512 0.001  0.178 ± 0.007  
    fragSMILES 2  512 512 0.001  0.169 ± 0.009  
    t-SMILES   2  512 512 0.001    0.16 ± 0.01  
5   SELFIES    2  512 256 0.001    0.17 ± 0.01  
    SMILES     2  512 256 0.001    0.19 ± 0.02  
    fragSMILES 2  512 256 0.001    0.19 ± 0.02  
    t-SMILES   2  512 256 0.001    0.17 ± 0.01

In [ ]:
scaffStats.to_csv('results/08_grisoniScaffMetrics.csv')

In [21]:
params=[2,512,512,0.001]
params5=[2,512,256,0.001]

In [22]:
genPvals = pd.DataFrame.from_dict(
                {notation+'x1': 
                {metric:stats.ttest_ind(
                scaffGrp.get_group((1,notation,*params))[metric],
                scaffGrp.get_group((1,'fragSMILES',*params))[metric],
                ).pvalue for metric in scaffStats.columns} for notation in ['SMILES','SELFIES']
                } |
                {notation+'x5': 
                {metric:stats.ttest_ind(
                scaffGrp.get_group((5,notation,*params5))[metric],
                scaffGrp.get_group((5,'fragSMILES',*params5))[metric],
                ).pvalue for metric in scaffStats.columns} for notation in ['SMILES','SELFIES']
                }
                , orient='index'
            )

In [23]:
genPvals

,total,unique,novel,chiral,unique%,novel%,chiral%
SMILESx1,0.122287,0.883691,2.905847e-02,0.042347,0.966772,3.886867e-04,0.125945
SELFIESx1,0.001574,0.034050,2.212652e-07,0.642110,0.007221,3.270683e-09,0.005719
SMILESx5,0.544191,0.070134,1.120501e-04,0.116053,0.064472,4.878120e-06,0.846064
SELFIESx5,1.000000,0.017568,2.491563e-06,0.417392,0.017827,9.042331e-08,0.042876


In [24]:
genPvals.map(lambda x: x<0.05)

,total,unique,novel,chiral,unique%,novel%,chiral%
SMILESx1,False,False,True,True,False,True,False
SELFIESx1,True,True,True,False,True,True,True
SMILESx5,False,False,True,False,False,True,False
SELFIESx5,False,True,True,False,True,True,True


## Count how many scaffolds are novel cause of new cyclic fragments

In [44]:
from src.processer import fragmentSmiles, applyFncPool

In [53]:
def checkScaffoldFragments(scaffolds, vocab):

    def checkNewFrag(frags):
        mask = np.in1d(frags, vocab)

        ## If u want track which new fragments are
        # return frags.values[~mask]
        return ~mask.any()
    
    # try:
    scaffFrags = pd.Series( applyFncPool(scaffolds, fnc=fragmentSmiles) ).str.split(' ')
    maskNewFrags=scaffFrags.apply(checkNewFrag)

    return maskNewFrags.sum()


In [43]:
vocabs=[]
train_data = pd.read_csv(f'data/chembl.tar.xz', usecols = ['fragsmiles'] + [f'fold{fold}' for fold in range(5)], compression="xz")
train_data['fragsmiles'] = train_data['fragsmiles'].str.split(' ')
for fold in range(5):
    query = f'fold{fold} == "train"'
    fold_data = train_data.query(query)['fragsmiles']
    vocabs.append(np.unique(np.concatenate(fold_data.values)))

In [54]:
def countNewScaffCauseFrags(evaluator):
    results = evaluator.novels.copy()

    results['inducedNewFrags']=results.apply(lambda x: checkScaffoldFragments(x.sampled.novelScaff, vocabs[x.fold]), axis=1)

    setup = {k:v for k,v in evaluator.setupArgs.items() if k!='es'}
    results = results.assign(**evaluator.datasetArgs, **setup)

    results.drop(columns='sampled', inplace=True)
    return results.sort_values(['amount','fold','epoch'])

In [51]:
inducedNewFrags=pd.concat( [ countNewScaffCauseFrags(evaluator) for evaluator in evaluators], ignore_index=True)

In [52]:
inducedNewFrags

,fold,amount,epoch,T,inducedNewFrags
0,0,6000,3,1.0,2371
1,1,6000,3,1.0,2235
2,2,6000,3,1.0,2345
3,3,6000,3,1.0,2293
4,4,6000,3,1.0,2253
5,0,6000,3,1.0,1029
6,1,6000,3,1.0,1075
7,2,6000,3,1.0,1073
8,3,6000,3,1.0,1122
9,4,6000,3,1.0,1013


In [84]:
induced = scaffDFs.merge(inducedNewFrags, on=compilerPatterns.index.tolist())

In [86]:
induced['inducedNewFrags%']=induced['inducedNewFrags']/induced['novel']

In [87]:
inducedGrp=induced.fillna(-1).groupby(compilerPatterns.index.drop('fold').tolist(),as_index=True, dropna=False)

/tmp/ipykernel_1141590/238393982.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  inducedGrp=induced.fillna(-1).groupby(compilerPatterns.index.drop('fold').tolist(),as_index=True, dropna=False)


In [88]:
inducedStats=inducedGrp.describe().loc(axis=1)[ ( slice(None),['mean','std'] ) ].drop(columns='fold')

In [89]:
inducedStats=ResultsFromMeanStdDF(inducedStats)

In [ ]:
inducedStats